# Question Generation을 위한 Keyword Extraction

- kiwi 형태소 분석기와 keyBERT를 사용
- train data 3,952개에서 augmentation 진행
- [Keybert와 kiwi형태소분석기를 사용하여 키워드추출 하기](https://hmkim312.github.io/posts/Keybert%EC%99%80_kiwi%ED%98%95%ED%83%9C%EC%86%8C%EB%B6%84%EC%84%9D%EA%B8%B0%EB%A5%BC_%EC%82%AC%EC%9A%A9%ED%95%98%EC%97%AC_%ED%82%A4%EC%9B%8C%EB%93%9C%EC%B6%94%EC%B6%9C_%ED%95%98%EA%B8%B0/) 참고 

## 1.데이터 준비

In [9]:
import random
import pandas as pd 
from datasets import load_from_disk

from keybert import KeyBERT
from kiwipiepy import Kiwi
from transformers import BertModel

In [8]:
# dataset
dataset = load_from_disk("../data/train_dataset/")

train_dataset = pd.DataFrame(dataset['train'])
val_dataset = pd.DataFrame(dataset['validation'])

In [10]:
model = BertModel.from_pretrained('skt/kobert-base-v1')
kw_model = KeyBERT(model)
kiwi = Kiwi()

In [11]:
# {'answer_start': [235], 'text': ['하원']}	형태로 되어있는 answers를 분리 해서 새로운 dataframe 생성
def split_answers(dataset):
    answer_start, text = [], []

    for i in range(len(dataset)):
        df = dataset.iloc[i]
        answer_start.append(df['answers']['answer_start'][0])
        answer = df['answers']['text'][0]
        text.append(answer)


    new_dataset = pd.DataFrame({'title': dataset['title'], 
                            'context' : dataset['context'], 
                            'question': dataset['question'], 
                            'id': dataset['id'],
                            'answer': text,
                            'answer_start': answer_start, 
                            'document_id': dataset['document_id'], 
                            '__index_level_0__': dataset['__index_level_0__'],})
    
    return new_dataset

In [12]:
train_df = split_answers(train_dataset)

In [13]:
train_df.tail(3)

,title,context,question,id,answer,answer_start,document_id,__index_level_0__
3949,빈 필하모니 관현악단,1842년에 작곡가이자 지휘자인 오토 니콜라이가 빈 궁정 오페라극장 소속 관현악단을...,1854년에 니콜라이의 뒤를 이어 상임 지휘자로서 활동한 인물은?,mrc-0-003429,칼 에케르트,255,12730,2252
3950,드래곤,원어는 고대 그리스어까지 거슬러 올라간다. 영어 문헌에 이 말이 나타나기 시작한 것...,드래곤의 암수 구분이 있는 신화는?,mrc-0-003956,슬라브 신화,476,10924,2595
3951,2011년 함부르크 주의회 선거,2008년 2월 28일 실시된 2008년 함부르크 주의회 선거에서 기민련은 과반수 ...,올레 폰 보이스트 이후에 시장으로 임명된 사람은 누구인가?,mrc-0-003589,크리스토포 알하우스,189,27803,2354


In [20]:
train_df.describe(include='all')

,title,context,question,id,answer,answer_start,document_id,__index_level_0__
count,3952,3952,3952,3952,3952,3952.000000,3952.000000,3952.000000
unique,2716,3340,3952,3952,3622,NaN,NaN,NaN
top,윤치호,"그러나 한편으로는 미국 사회의 인종차별주의적, 흑인을 차별하는 태도를 목격하면서 백...",대통령을 포함한 미국의 행정부 견제권을 갖는 국가 기관은?,mrc-1-000067,프랑스,NaN,NaN,NaN
freq,43,4,1,1,15,NaN,NaN,NaN
mean,NaN,NaN,NaN,NaN,NaN,376.794028,28399.783401,1496.027328
std,NaN,NaN,NaN,NaN,NaN,309.122555,17475.051830,1060.480948
min,NaN,NaN,NaN,NaN,NaN,0.000000,4572.000000,0.000000
25%,NaN,NaN,NaN,NaN,NaN,138.000000,11615.750000,587.000000
50%,NaN,NaN,NaN,NaN,NaN,310.000000,26295.000000,1189.500000
75%,NaN,NaN,NaN,NaN,NaN,538.000000,44240.250000,2416.250000


In [14]:
# 명사 추출
def noun_extractor(text):
    results = []
    result = kiwi.analyze(text)
    for token, pos, _, _ in result[0][0]:
        if len(token) != 1 and pos.startswith('N') or pos.startswith('SL'):
            results.append(token)
    return results

In [15]:
# 명사만 키워드 추출
def keyword_extraction_noun(context):
    nouns = noun_extractor(context)
    text = ' '.join(nouns) # 문자열 형태로 바꾸기 위해 join 
    keywords = kw_model.extract_keywords(text, keyphrase_ngram_range=(1, 1), stop_words=None, top_n=10)
    return keywords

In [20]:
# 형태소분석기 없이 키워드 추출
def keyword_extraction(context):
    keywords = kw_model.extract_keywords(context, keyphrase_ngram_range=(1, 3), stop_words=None, top_n=10)
    return keywords

## 1.샘플데이터 테스트

### 1_1. kiwi 형태소 분석기를 사용한 키워드 추출

In [16]:
sample_context = train_df['context'][1]
sample_answer = train_df['answer'][1]

In [17]:
sample_context

"'근대적 경영학' 또는 '고전적 경영학'에서 현대적 경영학으로 전환되는 시기는 1950년대이다. 2차 세계대전을 마치고, 6.25전쟁의 시기로 유럽은 전후 재건에 집중하고, 유럽 제국주의의 식민지가 독립하여 아프리카, 아시아, 아메리카 대륙에서 신생국가가 형성되는 시기였고, 미국은 전쟁 이후 경제적 변화에 기업이 적응을 해야 하던 시기였다. 특히 1954년 피터 드러커의 저서 《경영의 실제》는 현대적 경영의 기준을 제시하여서, 기존 근대적 인사조직관리를 넘어선 현대적 인사조직관리의 전환점이 된다. 드러커는 경영자의 역할을 강조하며 경영이 현시대 최고의 예술이자 과학이라고 주장하였고 , 이 주장은 21세기 인사조직관리의 역할을 자리매김했다.\\n\\n현대적 인사조직관리와 근대 인사조직관리의 가장 큰 차이는 통합이다. 19세기의 영향을 받던 근대적 경영학(고전적 경영)의 흐름은 기능을 강조하였지만, 1950년대 이후의 현대 경영학은 통합을 강조하였다. 기능이 분화된 '기계적인 기업조직' 이해에서 다양한 기능을 인사조직관리의 목적, 경영의 목적을 위해서 다양한 분야를 통합하여 '유기적 기업 조직' 이해로 전환되었다. 이 통합적 접근방식은 과정, 시스템, 상황을 중심으로 하는 인사조직관리 방식을 형성했다."

In [18]:
sample_answer

'《경영의 실제》'

In [61]:
result = keyword_extraction(sample_context)
result

[('경영학', 0.5645),
 ('경영', 0.561),
 ('년대', 0.5243),
 ('현대', 0.5128),
 ('경영자', 0.5021),
 ('세계', 0.4906),
 ('아시아', 0.4803),
 ('기업', 0.48),
 ('관리', 0.4795),
 ('상황', 0.4794)]

### 1_2. 형태소분석기 없이 키워드 추출

In [21]:
sample = train_df[:5]
for i in range(len(sample)):
    context = sample['context'][i]
    print(context)
    print(keyword_extraction(context))
    print()

미국 상의원 또는 미국 상원(United States Senate)은 양원제인 미국 의회의 상원이다.\n\n미국 부통령이 상원의장이 된다. 각 주당 2명의 상원의원이 선출되어 100명의 상원의원으로 구성되어 있다. 임기는 6년이며, 2년마다 50개주 중 1/3씩 상원의원을 새로 선출하여 연방에 보낸다.\n\n미국 상원은 미국 하원과는 다르게 미국 대통령을 수반으로 하는 미국 연방 행정부에 각종 동의를 하는 기관이다. 하원이 세금과 경제에 대한 권한, 대통령을 포함한 대다수의 공무원을 파면할 권한을 갖고 있는 국민을 대표하는 기관인 반면 상원은 미국의 주를 대표한다. 즉 캘리포니아주, 일리노이주 같이 주 정부와 주 의회를 대표하는 기관이다. 그로 인하여 군대의 파병, 관료의 임명에 대한 동의, 외국 조약에 대한 승인 등 신속을 요하는 권한은 모두 상원에게만 있다. 그리고 하원에 대한 견제 역할(하원의 법안을 거부할 권한 등)을 담당한다. 2년의 임기로 인하여 급진적일 수밖에 없는 하원은 지나치게 급진적인 법안을 만들기 쉽다. 대표적인 예로 건강보험 개혁 당시 하원이 미국 연방 행정부에게 퍼블릭 옵션(공공건강보험기관)의 조항이 있는 반면 상원의 경우 하원안이 지나치게 세금이 많이 든다는 이유로 퍼블릭 옵션 조항을 제외하고 비영리건강보험기관이나 보험회사가 담당하도록 한 것이다. 이 경우처럼 상원은 하원이나 내각책임제가 빠지기 쉬운 국가들의 국회처럼 걸핏하면 발생하는 의회의 비정상적인 사태를 방지하는 기관이다. 상원은 급박한 처리사항의 경우가 아니면 법안을 먼저 내는 경우가 드물고 하원이 만든 법안을 수정하여 다시 하원에 되돌려보낸다. 이러한 방식으로 단원제가 빠지기 쉬운 함정을 미리 방지하는 것이다.날짜=2017-02-05
[('united states senate', 0.7272), ('states senate', 0.721), ('상원의원으로 구성되어 있다', 0.7181), ('senate 양원제인 미국', 0.7094), ('senate', 0.6715), 

## 2. 키워드 추출 및 새로운 데이터 만들기

In [93]:
new_title = []
new_context = []
new_question = []
new_answer = []
new_answer_start = []
new_id = []
new_document_id = []
new_index_level = []

id_number = 3954
for idx in range(len(train_df)):
    keywords = keyword_extraction_noun(train_df.context[idx])
    for word in keywords:
        if word[0] != train_df.answer[idx] and len(word[0]) != 2 and word[1] > 0.6 and train_df.context[idx].count(word[0]) == 1:
            new_title.append(train_df.title[idx])
            new_context.append(train_df.context[idx])
            new_question.append('')
            new_answer.append(word[0])
            new_answer_start.append(train_df.context[idx].index(word[0]))
            id_name = 'mrc-1-00' + str(id_number)
            new_id.append(id_name)
            new_document_id.append(train_df.document_id[idx])
            new_index_level.append(id_number)
            id_number += 1

ke_data = pd.DataFrame({'title': new_title, 
                        'context' : new_context, 
                        'question': new_question, 
                        'id': new_id,
                        'answer': new_answer,
                        'answer_start': new_answer_start, 
                        'document_id': new_document_id, 
                        '__index_level_0__': new_index_level,})

In [94]:
ke_data.head(5)

,title,context,question,id,answer,answer_start,document_id,__index_level_0__
0,미국 상원,미국 상의원 또는 미국 상원(United States Senate)은 양원제인 미국...,,mrc-1-003954,공무원,268,18293,3954
1,계사명 사리구,동아대학교박물관에서 소장하고 있는 계사명 사리구는 총 4개의 용기로 구성된 조선후기...,,mrc-1-003955,웅촌면,62,47334,3955
2,계사명 사리구,동아대학교박물관에서 소장하고 있는 계사명 사리구는 총 4개의 용기로 구성된 조선후기...,,mrc-1-003956,울주군,58,47334,3956
3,오싱,하지만 슬픈 일들이 그녀의 행복을 질시하고 있었다.\n평생 고생만 한 어머니를 백혈...,,mrc-1-003957,가가야,64,10426,3957
4,오싱,하지만 슬픈 일들이 그녀의 행복을 질시하고 있었다.\n평생 고생만 한 어머니를 백혈...,,mrc-1-003958,아가씨,540,10426,3958


In [95]:
print(f'keywords extraction data TOTAL : ', len(ke_data))

keywords extraction data TOTAL :  3883


In [82]:
# 가중치 0.8 이상 => 29개 추출
ke_data_eight = ke_data
ke_data_eight.head(5)

,title,context,question,id,answer,answer_start,document_id,__index_level_0__
0,여운형,그의 죽음으로 좌우합작위원회는 구심점을 잃게 된다. 제2차 미소공위는 추진하던 중 ...,,mrc-1-003954,조선민주주의인민공화국,1098,5609,3954
1,피아노 협주곡 5번 (베토벤),"1802년으로 거슬러 올라가, 청각 장애를 우려해 ""하이리겐슈타트의 유서""를 쓴 베...",,mrc-1-003955,하이리겐슈타트의,29,7872,3955
2,8.3 인민소비품생산운동,이 8.3 인민소비품 생산 규정으로 인하여 1989년부터 현재인 2020년까지 한번...,,mrc-1-003956,조선민주주의인민공화국,155,57819,3956
3,강중인,경상북도 출신으로 1935년에 경성제국대학 법문학부 법학과를 졸업했다. 1938년에...,,mrc-1-003957,조선민주주의인민공화국,1196,15575,3957
4,양산 통도사 창녕포교당 목조석가여래좌상,통도사 창녕 포교당 법당에 모셔져 있는 나무로 만든 석가여래좌상이다. 이 불상은 원...,,mrc-1-003958,석가여래,29,47337,3958


In [92]:
# 가중치 0.7 이상 => 608개 추출
ke_data_seven = ke_data
ke_data_seven.head(5)

,title,context,question,id,answer,answer_start,document_id,__index_level_0__
0,모후산,모후산(母后山)대한민국 전라남도 화순군과 순천시의 경계를 이루는 높이 943.7m의...,,mrc-1-003954,대한불교조계종,374,15766,3954
1,모후산,모후산(母后山)대한민국 전라남도 화순군과 순천시의 경계를 이루는 높이 943.7m의...,,mrc-1-003955,조선로동당,402,15766,3955
2,모후산,모후산(母后山)대한민국 전라남도 화순군과 순천시의 경계를 이루는 높이 943.7m의...,,mrc-1-003956,조선말,1409,15766,3956
3,모후산,모후산(母后山)대한민국 전라남도 화순군과 순천시의 경계를 이루는 높이 943.7m의...,,mrc-1-003957,김삿갓,571,15766,3957
4,터미널 (2004년 영화),동유럽의 소국 크라코지아에서 온 빅토르 나보르스키가 미국 뉴욕의 존 F. 케네디 국...,,mrc-1-003958,국제공항,45,12023,3958


In [96]:
# 가중치 0.6 이상 => 3883개 추출
ke_data_six = ke_data
ke_data_six.head(5)

,title,context,question,id,answer,answer_start,document_id,__index_level_0__
0,미국 상원,미국 상의원 또는 미국 상원(United States Senate)은 양원제인 미국...,,mrc-1-003954,공무원,268,18293,3954
1,계사명 사리구,동아대학교박물관에서 소장하고 있는 계사명 사리구는 총 4개의 용기로 구성된 조선후기...,,mrc-1-003955,웅촌면,62,47334,3955
2,계사명 사리구,동아대학교박물관에서 소장하고 있는 계사명 사리구는 총 4개의 용기로 구성된 조선후기...,,mrc-1-003956,울주군,58,47334,3956
3,오싱,하지만 슬픈 일들이 그녀의 행복을 질시하고 있었다.\n평생 고생만 한 어머니를 백혈...,,mrc-1-003957,가가야,64,10426,3957
4,오싱,하지만 슬픈 일들이 그녀의 행복을 질시하고 있었다.\n평생 고생만 한 어머니를 백혈...,,mrc-1-003958,아가씨,540,10426,3958


In [ ]:
ke_data.to_csv('KEdata_for_generation_v2.csv', index=False)